# REVIEWS SCRAPING

In [109]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException, ElementClickInterceptedException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time, requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.options import Options
from time import sleep
from datetime import datetime
import pandas as pd
import re
import string

# Scrape TrustPilot 

~23 pages. Need to scrape a page, and move to the next one. Repeat until reach last page.

In [138]:
df = pd.DataFrame(columns=('name', 'date', 'rating', 'text', 'boaDate', 'boaText', 'source', 'responded', 'timeRetrieved', 'htmlId'))

In [139]:
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())
url = 'https://www.trustpilot.com/review/www.bankofamerica.com'

temp = 2
driver.implicitly_wait(5)
driver.maximize_window()
driver.get(url)
xpath1 = '/html/body/div[4]/div[3]/div/div/div[3]/button'
xpath2 = '/html/body/div[3]/a'
# close accept cookies tab
driver.implicitly_wait(15)

# close Covid notification tab
try:
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, xpath2))).click()
except:
    pass
sleep(2)
try:
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, xpath1))).click()
except TimeoutException:
    pass
sleep(2)
driver.implicitly_wait(15)

# close Covid notification tab
try:
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, xpath2))).click()
except:
    pass
    
sleep(2)
while True:   
    try:
        
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')

        for block in soup.find_all("article", class_='review'):
            htmlId = block.get('id')
            timeRetrieved = datetime.now()
            date = block.find('time', class_='review-date--tooltip-target').get('datetime').split('T')[0]
            name = block.find('div', class_='consumer-information__name').text.strip()
            stars = block.find('div', class_='star-rating star-rating--medium').find('img').get('alt').split(' ')[0]
            text = block.find('p', class_='review-content__text')
            if text:
                text = text.text.strip()
            else:
                text = block.find('a', class_='link link--large link--dark').text.strip()

            df = df.append({'name' : name , 'date' : date, 'rating': stars, 'text': text, 'boaDate': None, \
                            'boaText': None, 'source': 'trustpilot.com', 'timeRetrieved':timeRetrieved, \
                            'htmlId': htmlId, 'responded':False} , ignore_index=True)        

        next_page_btn = driver.find_element_by_partial_link_text("Next page")
        # open next page
        try:
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.PARTIAL_LINK_TEXT, 'Next page'))).click()
        except  StaleElementReferenceException:
            sleep(5)
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.PARTIAL_LINK_TEXT, 'Next page'))).click()
        print('.........Opened page', temp)
        temp += 1
        sleep(5)
    except:
        print("No more pages left")
        break
print('processed all pages')        


[WDM] - Driver [C:\Users\mskac\.wdm\drivers\geckodriver\win64\v0.28.0\geckodriver.exe] found in cache


.........Opened page 2
.........Opened page 3
.........Opened page 4
.........Opened page 5
.........Opened page 6
.........Opened page 7
.........Opened page 8
.........Opened page 9
.........Opened page 10
.........Opened page 11
.........Opened page 12
.........Opened page 13
.........Opened page 14
.........Opened page 15
.........Opened page 16
.........Opened page 17
.........Opened page 18
.........Opened page 19
.........Opened page 20
.........Opened page 21
.........Opened page 22
No more pages left
processed all pages


In [137]:
df.head()

,name,date,rating,text,boaDate,boaText,source,responded,timeRetrieved,htmlId
0,Calvin Johnson,2020-12-11,1,This is the worst merchant to use for merchant...,None,None,trustpilot.com,NaN,2020-12-11 23:21:44.000280,5fd3b52f755dc10b481f95e2
1,Rebecca Blackerby,2020-12-09,1,I went to check my balance and it said online ...,None,None,trustpilot.com,NaN,2020-12-11 23:21:44.004265,5fd1500b755dc10b481d66a4
2,Kahari Blackston,2020-12-09,1,They do not care or treat their customers with...,None,None,trustpilot.com,NaN,2020-12-11 23:21:44.008257,5fd0d8d3755dc107e0bd0797
3,Lia,2020-12-08,5,I can honestly say I've never had a bad experi...,None,None,trustpilot.com,NaN,2020-12-11 23:21:44.014204,5fcfebad755dc107e0bc4a60
4,D.A T,2020-12-07,2,Had a bad experience with being transferred to...,None,None,trustpilot.com,NaN,2020-12-11 23:21:44.018231,5fcea8c2755dc105f02d03aa


In [140]:
df.to_csv('data/data1.csv')

# Scrape BBB.com

A single page, but needs 'load more' button to be pressed until scroll to the end

In [141]:
# press load more until reach end of the page
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())
url = 'https://www.bbb.org/us/nc/charlotte/profile/bank/bank-of-america-0473-100421/customer-reviews'
driver.implicitly_wait(5)
driver.maximize_window()
driver.get(url)

xpath = '/html/body/div[2]/div[2]/div/button'
css_selector = 'button.MuiButton-root:nth-child(2)'


try:
    # close cookies notification
    myElem = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, xpath)))
    myElem.click()
except TimeoutException:
    pass
driver.implicitly_wait(15)

while True:   
    try:   
        # load more reviews
        myElem = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, css_selector)))
        myElem.click()
        print('.........Loaded more reviews')
        sleep(5) # otherwise spinning circle blocks the button
    except TimeoutException:
        print("Done loading reviews")
        break    

[WDM] - Driver [C:\Users\mskac\.wdm\drivers\geckodriver\win64\v0.28.0\geckodriver.exe] found in cache


.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loade

In [142]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

In [143]:
reviews = []
for block in soup.find_all("div", class_='MuiGrid-root styles__Review-sc-1azxajg-0 fyMiFZ dtm-review MuiGrid-container'):
    stars = block.find_all(d='M259.3 17.8L194 150.2 47.9 171.5c-26.2 3.8-36.7 36.1-17.7 54.6l105.7 103-25 145.5c-4.5 26.3 23.2 46 46.4 33.7L288 439.6l130.7 68.7c23.2 12.2 50.9-7.4 46.4-33.7l-25-145.5 105.7-103c19-18.5 8.5-50.8-17.7-54.6L382 150.2 316.7 17.8c-11.7-23.6-45.6-23.9-57.4 0z')
    stars = len(stars)
    date = block.find("p", class_='MuiTypography-root Typography-y2r0fa-0 kpIiVF MuiTypography-body2').text
    name = block.find('p', class_='MuiTypography-root Name-t42m9k-0 kSwwPu MuiTypography-body2').text
    text = block.find_all('div', class_='MuiTypography-root Text-sc-12c66pm-0 fgbKlJ MuiTypography-body2')
    timeRetrieved = datetime.now()
    responded = False
    
    if len(text)>1:
        boaDate = block.find('p', class_='MuiTypography-root Date-sc-8slhbi-0 kEubpt MuiTypography-body1').text
        boaText = text[1].text
        responded = True
    else:
        boaDate = None
        boaText = None
        
    review = text[0].text
    df = df.append({'name' : name , 'date' : date, 'rating': stars, 'text': review, 'boaDate': boaDate, \
                            'boaText': boaText, 'source': 'bbb.org', 'timeRetrieved':timeRetrieved, \
                            'htmlId': name+review[:10], 'responded':responded} , ignore_index=True) 
    

In [147]:
df.head()

,name,date,rating,text,boaDate,boaText,source,responded,timeRetrieved,htmlId
0,Calvin Johnson,2020-12-11,1,This is the worst merchant to use for merchant...,None,None,trustpilot.com,False,2020-12-11 23:24:47.991795,5fd3b52f755dc10b481f95e2
1,Rebecca Blackerby,2020-12-09,1,I went to check my balance and it said online ...,None,None,trustpilot.com,False,2020-12-11 23:24:48.002770,5fd1500b755dc10b481d66a4
2,Kahari Blackston,2020-12-09,1,They do not care or treat their customers with...,None,None,trustpilot.com,False,2020-12-11 23:24:48.006710,5fd0d8d3755dc107e0bd0797
3,Lia,2020-12-08,5,I can honestly say I've never had a bad experi...,None,None,trustpilot.com,False,2020-12-11 23:24:48.012692,5fcfebad755dc107e0bc4a60
4,D.A T,2020-12-07,2,Had a bad experience with being transferred to...,None,None,trustpilot.com,False,2020-12-11 23:24:48.017728,5fcea8c2755dc105f02d03aa


# Scrape DepositAccounts.com

1. Click 'View more'
2. Find and click all 'Read More'
3. Result: single page with reviews, so just scrape it

In [149]:
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())
url = 'https://www.depositaccounts.com/banks/bank-of-america.html'

driver.implicitly_wait(5)
driver.maximize_window()
driver.get(url)

[WDM] - Driver [C:\Users\mskac\.wdm\drivers\geckodriver\win64\v0.28.0\geckodriver.exe] found in cache


In [150]:
readMoreClassName = 'textExpand'

# click 'View more'
try:    
    viewMoreBtn = driver.find_element_by_partial_link_text("View MORE")
    viewMoreBtn.click()
except TimeoutException:
    print("Couldn't open all reviews page")

sleep(10)

# expand all reviews
more_buttons = driver.find_elements_by_class_name(readMoreClassName)
for x in range(len(more_buttons)):
    if more_buttons[x].is_displayed():
        driver.execute_script("arguments[0].click();", more_buttons[x])
        time.sleep(1)
        

In [151]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')
starClassRe = re.compile('^stars')

for block in soup.find_all("div", class_='bankReviewContainer'):
    title = block.find('h3').text
    
    stars = block.find('div', {"class" : starClassRe}).get('class')
    stars = int(stars[1][-1])
    
    name = block.find('span', itemprop='author').text
    date = block.find('span', itemprop='datePublished').get('datetime')
    y, m, d = date.split('-')
    date = '-'.join([y, m, d])
    
    text = block.find('p', itemprop='description').text
    review = title+' '+text
    
    timeRetrieved = datetime.now()
    htmlId = block.find('div', class_='bankReview').get('id')
    
    df = df.append({'name' : name , 'date' : date, 'rating': stars, 'text': review, 'boaDate': None, \
                            'boaText': None, 'source': 'depositaccounts.com', 'timeRetrieved':timeRetrieved, \
                            'htmlId': htmlId, 'responded':False} , ignore_index=True) 
    


In [153]:
df['city'] = None
df['state'] = None

# Scrape ConsumerAffairs.com

1. Click read full review btn
2. Process data
3. Click Next btn
4. Repeat 1-3 until no Next btn

In [174]:
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())
url = 'https://www.consumeraffairs.com/finance/bofa.html'

driver.implicitly_wait(5)
driver.maximize_window()
driver.get(url)

[WDM] - Driver [C:\Users\mskac\.wdm\drivers\geckodriver\win64\v0.28.0\geckodriver.exe] found in cache


In [175]:
monthVocab = {'jan': 1, 'feb': 2, 'march':3 , 'april':4, 'may': 5, 'june':6, 'july':7 , 'aug':8, 'sept':9 ,'oct':10, 'nov': 11,'dec':12}
page = 2

while True:   
    try:
        
        #expend all reviews
        more_buttons = driver.find_elements_by_partial_link_text('Read full review')

        for x in range(len(more_buttons)):
            if more_buttons[x].is_displayed():
                driver.execute_script("arguments[0].click();", more_buttons[x])
                time.sleep(1)
        # process data on current page        
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')

        for block in soup.find_all("div", class_='rvw js-rvw'):

            stars = block.find('meta', itemprop = 'ratingValue').get('content')
            stars = int(stars)

            user = block.find('strong', itemprop = 'author').text
            name = user.split(' of ')
            if len(user) == 1:
                city = state = None
            else:
                cityState = user[1].split(', ')
                if len(cityState) ==1:
                    if len(cityState[0])==2:
                        state = cityState[0]
                        city = None
                    else: 
                        state = None
                        city = cityState[0]
                else:
                    city, state = user[1].split(', ')

            date = block.find('span', class_ = 'ca-txt-cpt').text.split(': ')[1]
            date = date.replace('.', '').replace(',', '').lower()
            m, d, y = date.split(' ')
            m = str(monthVocab[m])
            date = '-'.join([y, m.zfill(2), d.zfill(2)])

            text = block.find('div', class_ = 'rvw-bd').find_all('p')[1].text
            timeRetrieved = datetime.now()
            htmlId = block.get('id')

            df = df.append({'name' : name , 'date' : date, 'rating': stars, 'text': text, 'boaDate': None, \
                                'boaText': None, 'source': 'consumeraffairs.com', 'timeRetrieved':timeRetrieved, \
                                'htmlId': htmlId, 'responded':False} , ignore_index=True)
        next_page_btn = driver.find_element_by_partial_link_text("Next")
        # open next page
        try:
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.PARTIAL_LINK_TEXT, 'Next'))).click()
        except  StaleElementReferenceException:
            sleep(5)
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.PARTIAL_LINK_TEXT, 'Next'))).click()
        print('.........Opened next page', page)
        page += 1
    except:
        print("No more pages left")
        break
print('processed all pages')    

review-8166833
review-8166908
review-8166809
review-8167259
review-7993835
review-8806574
review-8801513
review-8801282
review-8800697
review-8800682
.........Opened next page 2
review-8797511
review-8679860
review-8675762
review-8509448
review-8492447
review-8444279
review-8442821
review-8437478
review-8413013
review-8404271
.........Opened next page 3
review-8403074
review-8376941
review-8374334
review-8373365
review-8368160
review-8366780
review-8364428
review-8358977
review-8358377
review-8357189
.........Opened next page 4
review-8346818
review-8330729
review-8323055
review-8305934
review-8304221
review-8303993
review-8299622
review-8298221
review-8243642
review-8241533
review-8226404
review-8187119
review-8182466
review-8180462
review-8166941
review-8167079
review-8166719
review-8166986
review-8166785
review-8167058
review-8166920
review-8166794
review-8167064
review-8156105
review-8166755
review-8167019
review-8166863
review-8152013
review-8166773
review-8167043
.........Opened 

In [176]:
df.to_csv(r'data\reviewsDataset.csv')

In [177]:
len(df)

1669

# Scraping Wallethub.com

Using Scraping Robot saved locally all pages.
Scraping from local htmls.

In [35]:
from os import listdir
from os.path import isfile, join
import re

mypath = 'data\htmls'
files = [mypath+'\\'+ f for f in listdir(mypath) if isfile(join(mypath, f))]
print(len(files))
df = pd.DataFrame(columns=('name', 'date', 'stars', 'text', 'boaDate', 'boaText', 'product', 'source'))

244


In [36]:
for file in files:
    soup = BeautifulSoup(open(file,  encoding="utf8").read())

    for block in soup.find_all('article', itemprop = 'review'):
        user = block.find('span', itemprop='author').text
        date = block.find('time', class_='rvtab-ci-time regular-font').get('datetime')
        stars = block.find('meta', itemprop='ratingValue').get('content')
        text = block.find('div', itemprop='description').text
        product = block.find('div', class_='rvtab-ci-category')
        if product:
            product = product.text
            product = re.split('<.span>', product)
            product = product[0].split(': ')[1]
        else:
            product = None

        df = df.append({'name' : user.strip() , 'date' : date, 'stars': int(stars), 'text': text, \
                        'boaDate': None, 'boaText': None, 'product': product, 'source': 'wallethub.com'}\
                       , ignore_index=True) 

In [178]:
df_wallethub = pd.read_csv('data/rawScrapedData/wallethub_processed.csv')